# Extracting regional values from Met Office Global meteorological data

## Process
This notebook runs you through how to extract spatial mean values from gridded data using shapefiles. The process includes:
1. Loading the gridded data from NetCDF files into memory using Iris (using [lazy loading](https://scitools.org.uk/iris/docs/latest/userguide/real_and_lazy_data.html)).
2. Subset the global data to only include the USA, improving the processing time.
3. Load the Shapefile for the regions we want to subset with.
4. Define the functions to be used in the pipeline.
5. Loop through all the regions in the shapefile; subsetting, collapsing and saving out to a CSV file for each region.
6. Load all the region CSVs, collate into one large DataFrame and save out to CSV.

## Method
This process uses the polygon of a region (from the shapefile) to subset the gridded data by getting the **latitude-longitude bounding box** of the polygon, as described in this diagram:

<img src="images/coarse_spatial_mean_gridded.png" alt="Lat-Lon bounding box using polygon" style="height: 400px;"/> 

Each grid cell (small latitude-longitude box) contains a single value for a meteorological variable. The single value of that variable for the whole region/polygon is the mean of all the grid cell values in the bounding box i.e. lat-lon spatial mean.

For example, here we have air temperature values in a bounding box that covers the a polygon. The temperature value for the region is the mean value of the temperatures in the boundind box - 20.9°C.

<img src="images/spatial_mean_example.png" alt="The mean value for the temperature is 20.9°C" style="height: 400px;"/> 

#### Time
Of course we have ignored the time axis in this example, which is present in the gridded data but is handled for us by the Iris library as just another dimension. In this notebook we use daily data and will simply store the date for each value in the final tabular data.

#### Improvements
This process could be more accurate by only using the grid cells which actually overlap with the polygon and by weighting the grid cells according to how much of their area is within the polygon. Improvements like these are coming.

## Setup

In [ ]:
#Data
import iris
import cartopy.io.shapereader as shpreader
import pandas as pd
import cftime

#Plotting
import iris.plot as iplt
import iris.quickplot as qplt
import matplotlib.pyplot as plt
%matplotlib inline

#System
import os
import sys
import glob

#Met Office utils
import shape_utils as shape

#Supress warnings
import warnings
warnings.filterwarnings('ignore')

### 1. Load Met Office Global Data

The files for each variable are contained in a separate folder.

In [ ]:
#List all the filepaths and store in a dict with each variable as a key
folder = '/data/covid19-ancillary-data/latest/mo_data_global_daily/'
filepaths = {path: glob.glob(os.path.join(folder, path, '*.nc')) for path in os.listdir(folder)}
variables = list(filepaths.keys())

print(variables)
print(f'Number of files for each variable: {len(filepaths[variables[0]])}')

In [ ]:
%%time
#Run through all the variables and append the loaded cubes to a CubeList
cubes = iris.cube.CubeList([])

for var in variables:
    cubes.extend(iris.load(filepaths[var]))
    
print(cubes)

## 2. Subset global data to the country we want

In [ ]:
#Subset the cubes to just the USA
us_latlon = ((18, 75), (-179, -65))
us_cubes = iris.cube.CubeList([cube.intersection(latitude=us_latlon[0], longitude=us_latlon[1]) for cube in cubes])
print(us_cubes)

In [ ]:
#Plot the subset to check that we have the right area
qplt.contourf(us_cubes[0][0])
plt.gca().coastlines()

In [ ]:
#Extract the coordinate reference system from one of the cubes. We will use this later.
CRS = us_cubes[0].coord_system()
CRS

### 3. Load shapefile containing region polygons

In [ ]:
#Load the shapefile
shapefile = 'US_COUNTY_POP.shx'
regions_reader = shpreader.Reader(shapefile)

In [ ]:
#How many regions are included?
len([record for record in regions_reader.records()])

In [ ]:
#Let's take a look at one
next(regions_reader.records())

In [ ]:
next(regions_reader.geometries())

In [ ]:
#We will use a list of the region IDs to loop through later
region_ids = [record.attributes['OBJECTID'] for record in regions_reader.records()]

In [ ]:
print(region_ids[0], region_ids[-1])

In [ ]:
#Helper function to get the record from the reader
def get_region_record(target, shapefile=regions_reader, attribute='OBJECTID'):
    '''
    Get the geometries for the specified target.
    
    '''
    result = None
    for record in shapefile.records():
        location = record.attributes[attribute]
        if location == target:
            result = record
            break
    if result is None:
        emsg = f'Could not find region with {attribute} "{target}".'
        raise ValueError(emsg)
    return result

In [ ]:
#Create a random ID generator
from random import randint
def rand_id(ids=region_ids): 
    return randint(ids[0], ids[-1])

In [ ]:
#Get a random geometry to check it's all working as expected
get_region_record(regions_reader, rand_id()).geometry

## 4. Define the functions to be used in the pipeline.

In [ ]:
def parse_data_name(cube):
    name = cube.name()
    method = cube.cell_methods[0].method.replace('imum', '')
    units = cube.units
    
    if name == 'm01s01i202':
        name = 'short_wave_radiation'
        units = 'W/m2'
    
    return f'{name}_{method} ({units})'

def get_date(dt):
    if isinstance(dt, cftime.real_datetime):
        date = dt.date()
    else:
        try:
            date = datetime.datetime(dt.year, dt.month, dt.day).date()
        except e:
            raise Exception(e)
    return date

In [ ]:
def extract(location, cubes=us_cubes, **kwargs):
    region = get_region_record(location, **kwargs)
    cutter = shape.Shape(region.geometry, region.attributes, coord_system=CRS)
    cut_cubes = cutter.extract_subcubes(cubes)
#     cubes_col = [cube.collapsed(['latitude','longitude'], iris.analysis.MEAN) for cube in cut_cubes]
    return cut_cubes

In [ ]:
def extract_collapse_df(location, cubes=us_cubes, **kwargs):
    region = get_region_record(location, **kwargs)
    cutter = shape.Shape(region.geometry, region.attributes, coord_system=CRS)
    cut_cubes = cutter.extract_subcubes(cubes)
    cubes_col = [cube.collapsed(['latitude','longitude'], iris.analysis.MEAN) for cube in cut_cubes]
    time = cubes_col[0].coord('time')
    length = len(time.points)
    data = {'objectid': [location]*length,
            'fips': [region.attributes['FIPS']]*length,
            'county_name': [region.attributes['NAME']]*length,
            'state_name': [region.attributes['STATE_NAME']]*length,
            'date': [get_date(cell.point) for cell in time.cells()]}
    data.update({parse_data_name(cube): cube.data for cube in cubes_col})
    
    df = pd.DataFrame(data, columns=COL_ORDER)

    return df

In [ ]:
id_ = rand_id()
print(id_)
print(extract(id_, us_cubes, attribute='OBJECTID'))
display(get_region_record(regions_reader, id_, 'OBJECTID').geometry)

In [ ]:
df_ex = extract_collapse_df(id_)
df_ex

## 5. Loop through all the regions in the shapefile; subsetting, collapsing and saving out to a CSV file for each region.

In [ ]:
#Set the order of the columns in the dataframes we will create
col0 = ['objectid', 'fips', 'county_name', 'state_name', 'date']
col1 = [parse_data_name(cube) for cube in us_cubes]
COL_ORDER = tuple(col0 + sorted([c for c in col1 if c not in col0]))
COL_ORDER

In [ ]:
#Let's look at the cubes we are going to 
print(us_cubes)

In [ ]:
#For now let's assume we haven't written any files, so we will loop through all the region IDs
unwritten = region_ids

In [ ]:
%%time
#This will loop through all the region IDs, executing extract_collapse_df for each region and saving it to a CSV file
#Any errors will be caught and printed, but the loop will continue onto the next ID
#Note that we cannot write to covid19-ancillary-data, so will have to write to /data/share/
start = len(csvs)
stop = len(region_ids)
for location in unwritten[start:]:
    try:
        df = extract_collapse_df(location)
        fname = df['fips'][0]
        county = df['county_name'][0]
        state = df['state_name'][0]
        df.to_csv(f'/data/share/us_data/us_{fname}_daily_data_2020jan-mar.csv', index=False)
        print(f'  [{location}] {fname}, {county}, {state}: Success')
    except Exception as e:
        print(f'x [{location}] {fname}, {county}, {state}: Error \n  x  {e}')

## 6. Load all the region CSVs, collate into one large DataFrame and save out to CSV.

In [ ]:
#List all the csvs in /data/share/us_data/
csvs = glob.glob('/data/share/us_data/*.csv')
len(csvs)

In [ ]:
%%time
#Now load all the CSVs for each region and combine into one large dataframe
df = pd.concat([pd.read_csv(csv) for csv in csvs], ignore_index=True)
df

In [ ]:
#And save to a CSV
fname_write = '/data/share/us_daily_precipdata_2020jan-mar_v01.csv'
Mdf.to_csv(fname_write, index=False)

In [ ]:
#We can read it back in to check that it wrote correctly
pd.read_csv(fname_write)